In [ ]:
import win32gui
import win32ui
import win32con
from PIL import Image
import ctypes
from ctypes import wintypes
import cv2
import numpy as np
try:
    ctypes.windll.shcore.SetProcessDpiAwareness(2)  
except Exception as e:
    print("DPI 感知设置失败：", e)

In [27]:
title_keyword="《星际争霸II》"
# 找到窗口句柄
hwnd = win32gui.FindWindow(None, None)
while hwnd:
    if title_keyword in win32gui.GetWindowText(hwnd):
        break
    hwnd = win32gui.GetWindow(hwnd, win32con.GW_HWNDNEXT)

if not hwnd:
    print("未找到窗口")
#棋盘坐标
left = 1740
top = 134
right = 2508
bottom = 902
width = right - left
height = bottom - top
# 创建设备上下文
hwndDC = win32gui.GetWindowDC(hwnd)
mfcDC = win32ui.CreateDCFromHandle(hwndDC)
saveDC = mfcDC.CreateCompatibleDC()
# 创建位图
saveBitMap = win32ui.CreateBitmap()
saveBitMap.CreateCompatibleBitmap(mfcDC, width, height)
saveDC.SelectObject(saveBitMap)
# 截图（窗口即使被遮挡也能截）
saveDC.BitBlt((0, 0), (width, height), mfcDC, (left, top), win32con.SRCCOPY)
# 转换为PIL图像
bmpinfo = saveBitMap.GetInfo()
bmpstr = saveBitMap.GetBitmapBits(True)
img = Image.frombuffer('RGB', (bmpinfo['bmWidth'], bmpinfo['bmHeight']), bmpstr, 'raw', 'BGRX', 0, 1)
# 清理资源
win32gui.DeleteObject(saveBitMap.GetHandle())
saveDC.DeleteDC()
mfcDC.DeleteDC()
win32gui.ReleaseDC(hwnd, hwndDC)

0

In [28]:
img.show()

In [36]:
matrix = [
    [4, 6, 6, 2, 5, 4, 1, 5],
    [2, 5, 1, 1, 6, 1, 5, 1],
    [2, 3, 5, 3, 6, 6, 3, 5],
    [3, 5, 1, 3, 5, 6, 6, 5],
    [4, 1, 6, 5, 2, 1, 6, 1],
    [1, 2, 5, 4, 1, 3, 2, 4],
    [2, 6, 5, 6, 5, 2, 3, 6],
    [3, 4, 1, 2, 6, 2, 5, 2],
]
for i in range(8):
    for j in range(8):
        if j<7 :
            print(matrix[i][j], end=',')
        if i<7 :
            print(matrix[i][j], end=',')
    print('')

4,4,6,6,6,6,2,2,5,5,4,4,1,1,5,
2,2,5,5,1,1,1,1,6,6,1,1,5,5,1,
2,2,3,3,5,5,3,3,6,6,6,6,3,3,5,
3,3,5,5,1,1,3,3,5,5,6,6,6,6,5,
4,4,1,1,6,6,5,5,2,2,1,1,6,6,1,
1,1,2,2,5,5,4,4,1,1,3,3,2,2,4,
2,2,6,6,5,5,6,6,5,5,2,2,3,3,6,
3,4,1,2,6,2,5,


In [ ]:
import numpy as np

import recognize


def find_best_move(matrix: np.ndarray, simulations: int = 10) -> tuple[tuple[tuple[int, int], tuple[int, int]], int, int]:
    """
    找出能引发最长连锁的移动
    评分 = (连锁轮数 * 1000 + 总消除数)  # 权重确保连锁优先
    参数:
        matrix: 棋盘矩阵
        simulations: 每个移动的模拟次数
    返回:
        best_move: 最佳移动位置 ((r1, c1), (r2, c2))
        best_elim: 预计最大消除数量
        best_chain: 预计最大连锁轮数
    
    """
    rows, cols = len(matrix), len(matrix[0])
    best_move = ((0, 0), (0, 0))
    best_score = -1
    best_elim = 0
    best_chain = 0
    # 遍历所有可能的相邻交换
    for i in range(rows):
        for j in range(cols):
            # 向右交换
            if j < cols - 1:
                max_elim, max_chain = evaluate_move_expectation(matrix, i, j, i, j + 1, simulations)
                score = max_elim + max_chain * 1000  # 连锁优先

                if score > best_score:
                    best_score = score
                    best_move = ((i, j), (i, j + 1))
                    best_elim = max_elim
                    best_chain = max_chain

            # 向下交换
            if i < rows - 1:
                max_elim, max_chain = evaluate_move_expectation(matrix, i, j, i + 1, j, simulations)
                score = max_elim + max_chain * 1000

                if score > best_score:
                    best_score = score
                    best_move = ((i, j), (i + 1, j))
                    best_elim = max_elim
                    best_chain = max_chain

    return best_move, best_elim, best_chain


def simulate_swap(matrix, r1, c1, r2, c2, seed=None):
    """
    模拟交换并返回 (总消除数, 连锁轮数)
    增加 seed 参数以支持多次模拟
    """
    if seed is not None:
        np.random.seed(seed)

    board = [row[:] for row in matrix]  # 深拷贝
    board[r1][c1], board[r2][c2] = board[r2][c2], board[r1][c1]

    total_eliminated = 0
    chain_rounds = 0

    while True:
        eliminated_this_round = find_and_eliminate(board)
        if eliminated_this_round == 0:
            break
        total_eliminated += eliminated_this_round
        chain_rounds += 1
        simulate_fall(board)

    return total_eliminated, chain_rounds
simulate_swap(matrix, 0, 0, 0, 1, seed=42)

def evaluate_move_expectation(matrix: np.ndarray, r1: int, c1: int, r2: int, c2: int, simulations: int = 10) -> tuple[int, int]:
    """
    对一次移动进行多次模拟，返回最大连锁轮数和最大总消除
    参数:
        matrix: 棋盘矩阵
        r1, c1: 第一个方块位置
        r2, c2: 第二个方块位置
        simulations: 模拟次数
    返回:
        (max_eliminated, max_chain): 最大消除数量和最大连锁轮
    """

    max_chain = 0
    max_elim = 0
    for _ in range(simulations):
        elim, chain = simulate_swap(matrix, r1, c1, r2, c2, seed=None)  # 随机种子
        max_elim = max(max_elim, elim)
        max_chain = max(max_chain, chain)

    return max_elim, max_chain


def find_and_eliminate(board):
    """
    查找并消除水平和垂直方向上连续3个或以上的相同方块
    返回本次消除的数量
    """
    rows, cols = len(board), len(board[0])
    to_eliminate = set()

    # 水平方向检测
    for i in range(rows):
        j = 0
        while j < cols:
            if board[i][j] == 0:
                j += 1
                continue
            color = board[i][j]
            count = 1
            while j + count < cols and board[i][j + count] == color:
                count += 1
            if count >= 3:
                for k in range(count):
                    to_eliminate.add((i, j + k))
            j += count if count > 1 else 1

    # 垂直方向检测
    for j in range(cols):
        i = 0
        while i < rows:
            if board[i][j] == 0:
                i += 1
                continue
            color = board[i][j]
            count = 1
            while i + count < rows and board[i + count][j] == color:
                count += 1
            if count >= 3:
                for k in range(count):
                    to_eliminate.add((i + k, j))
            i += count if count > 1 else 1

    # 执行消除
    for r, c in to_eliminate:
        board[r][c] = 0

    return len(to_eliminate)


def simulate_fall(board):
    """
    模拟方块下落，并在顶部生成新的随机方块（1-6）
    """
    rows, cols = len(board), len(board[0])

    for j in range(cols):
        # 收集该列非零元素，从底部排列
        col_vals = []
        for i in range(rows):
            if board[i][j] != 0:
                col_vals.append(board[i][j])

        # 从底部填充
        for i in range(rows - 1, -1, -1):
            if col_vals:
                board[i][j] = col_vals.pop()
            else:
                board[i][j] = np.random.randint(1, 7)  # 新方块 1~6


def print_board(board, title="棋盘"):
    """打印棋盘状态"""
    print(f"{title}:")
    for i, row in enumerate(board):
        print(f"行{i}: " + "  ".join([f"{cell:2}" for cell in row]))


def visualize_move(matrix, move):
    """
    可视化一次移动的真实连锁过程
    """
    board = [row[:] for row in matrix]
    (r1, c1), (r2, c2) = move

    print(f"原始棋盘:")
    print_board(board)

    print(f"交换前: ({r1},{c1})={matrix[r1][c1]}, ({r2},{c2})={matrix[r2][c2]}")
    board[r1][c1], board[r2][c2] = board[r2][c2], board[r1][c1]
    print(f"交换后: ({r1},{c1})={board[r1][c1]}, ({r2},{c2})={board[r2][c2]}")
    print_board(board)

    total_eliminated = 0
    round_num = 1

    while True:
        eliminated_this_round = find_and_eliminate(board)
        if eliminated_this_round == 0:
            break
        print(f"\n第{round_num}轮消除: {eliminated_this_round}个方块")
        print_board(board, f"第{round_num}轮消除后")

        total_eliminated += eliminated_this_round

        simulate_fall(board)
        print_board(board, f"第{round_num}轮下落后")
        round_num += 1

    chain_count = round_num - 1
    print(f"\n总消除数量: {total_eliminated}, 连锁轮数: {chain_count}")
    return total_eliminated, chain_count


from PIL import Image, ImageDraw, ImageFont


def draw_best_move_on_board_image(img, best_move):
    """
    在 768x768 的棋盘图像上绘制最佳移动
    img: PIL.Image 对象，768x768，表示 8x8 棋盘
    best_move: ((r1, c1), (r2, c2))
    """
    # 创建可绘制对象
    draw = ImageDraw.Draw(img)

    # 字体（避免警告）
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except:
        font = ImageFont.load_default()

    # 颜色
    MARK_COLOR = (255, 0, 0)  # 红色标记
    ARROW_COLOR = (255, 255, 0)  # 黄色箭头

    cell_w, cell_h = 96, 96  # 768 / 8 = 96

    if best_move:
        (r1, c1), (r2, c2) = best_move

        # --- 1. 计算中心点 ---
        x1 = c1 * cell_w + cell_w // 2
        y1 = r1 * cell_h + cell_h // 2
        x2 = c2 * cell_w + cell_w // 2
        y2 = r2 * cell_h + cell_h // 2

        # --- 2. 画红圈标记 ---
        radius = 20
        draw.ellipse([(x1 - radius, y1 - radius), (x1 + radius, y1 + radius)], outline=MARK_COLOR, width=4, fill=None)
        draw.ellipse([(x2 - radius, y2 - radius), (x2 + radius, y2 + radius)], outline=MARK_COLOR, width=4, fill=None)

        # --- 3. 画双向箭头 ---
        # Pillow 不支持直接画箭头，我们用 line + text 模拟
        draw.line([(x1, y1), (x2, y2)], fill=ARROW_COLOR, width=5)

        # --- 4. 标注文字 ---
        draw.text((x1 - 40, y1 - 40), "Start", fill=MARK_COLOR, font=font, stroke_fill=(255, 255, 255), stroke_width=2)
        draw.text((x2 - 30, y2 - 40), "End", fill=MARK_COLOR, font=font, stroke_fill=(255, 255, 255), stroke_width=2)

    return img

In [44]:

def print_board(board, title="棋盘"):
    """打印棋盘状态"""
    print(f"{title}:")
    for i, row in enumerate(board):
        print(f" ".join([f"{cell:2}" for cell in row]))
        
def simulate_swap(matrix, r1, c1, r2, c2, seed=None):
    """
    模拟交换并返回 (总消除数, 连锁轮数)
    增加 seed 参数以支持多次模拟
    """
    if seed is not None:
        np.random.seed(seed)
    print_board(matrix)
    board = [row[:] for row in matrix]  # 深拷贝
    board[r1][c1], board[r2][c2] = board[r2][c2], board[r1][c1]
    print_board(board)
    total_eliminated = 0
    chain_rounds = 0

    while True:
        eliminated_this_round = find_and_eliminate(board)
        if eliminated_this_round == 0:
            break
        total_eliminated += eliminated_this_round
        chain_rounds += 1
        simulate_fall(board)

    return total_eliminated, chain_rounds
simulate_swap(matrix, 1, 2, 0, 2, seed=42)

棋盘:
 4  6  6  2  5  4  1  5
 2  5  1  1  6  1  5  1
 2  3  5  3  6  6  3  5
 3  5  1  3  5  6  6  5
 4  1  6  5  2  1  6  1
 1  2  5  4  1  3  2  4
 2  6  5  6  5  2  3  6
 3  4  1  2  6  2  5  2
棋盘:
 4  6  1  2  5  4  1  5
 2  5  6  1  6  1  5  1
 2  3  5  3  6  6  3  5
 3  5  1  3  5  6  6  5
 4  1  6  5  2  1  6  1
 1  2  5  4  1  3  2  4
 2  6  5  6  5  2  3  6
 3  4  1  2  6  2  5  2


(0, 0)

In [ ]:
def find_and_eliminate(board: np.ndarray) -> int:
    """
    查找并消除水平和垂直方向上连续3个或以上的相同方块
    返回本次消除的数量
    """
    rows, cols = len(board), len(board[0])
    to_eliminate = set()

    # 水平方向检测
    for i in range(rows):
        j = 0
        while j < cols:
            if board[i][j] == 0:
                j += 1
                continue
            color = board[i][j]
            count = 1
            while j + count < cols and board[i][j + count] == color:
                count += 1
            if count >= 3:
                for k in range(count):
                    to_eliminate.add((i, j + k))
            j += count if count > 1 else 1

    # 垂直方向检测
    for j in range(cols):
        i = 0
        while i < rows:
            if board[i][j] == 0:
                i += 1
                continue
            color = board[i][j]
            count = 1
            while i + count < rows and board[i + count][j] == color:
                count += 1
            if count >= 3:
                for k in range(count):
                    to_eliminate.add((i + k, j))
            i += count if count > 1 else 1

    # 执行消除
    for r, c in to_eliminate:
        board[r][c] = 0

    return len(to_eliminate)

In [47]:
def simulate_fall(board):
    """
    模拟方块下落，并在顶部生成新的随机方块（1-6）
    """
    rows, cols = len(board), len(board[0])

    for j in range(cols):
        # 收集该列非零元素，从底部排列
        col_vals = []
        for i in range(rows):
            if board[i][j] != 0:
                col_vals.append(board[i][j])

        # 从底部填充
        for i in range(rows - 1, -1, -1):
            if col_vals:
                board[i][j] = col_vals.pop()
            else:
                board[i][j] = np.random.randint(1, 7)  # 新方块 1~6
    
matrix = [
    [4, 6, 6, 2, 5, 4, 1, 5],
    [2, 5, 1, 1, 6, 1, 5, 1],
    [2, 3, 5, 3, 6, 6, 3, 5],
    [3, 5, 1, 3, 5, 6, 6, 5],
    [4, 1, 6, 0, 2, 1, 6, 1],
    [1, 2, 5, 0, 1, 3, 2, 4],
    [2, 6, 5, 0, 2, 2, 3, 6],
    [3, 4, 1, 2, 0, 2, 5, 2],
]
simulate_fall(matrix)
print_board(matrix)

棋盘:
 4  6  6  3  5  4  1  5
 2  5  1  3  5  1  5  1
 2  3  5  3  6  6  3  5
 3  5  1  2  6  6  6  5
 4  1  6  1  5  1  6  1
 1  2  5  3  2  3  2  4
 2  6  5  3  1  2  3  6
 3  4  1  2  2  2  5  2


In [7]:
mean_nums_count=[]
def check(nums):
    for i,n in enumerate(nums):
        if n==0:
            continue
        if i!=n:
            return False
    return True
def mean_nums(num):
    res=[0]*7
    while num:
        x=num%10
        if x>=1 and x<=6:
            res[x]+=1
        num=num//10
    return res
for i in range(10000000):
    tmp=mean_nums(i)
    # print(tmp)
    if check(tmp):
        mean_nums_count.append(i)
print(mean_nums_count)

[0, 1, 7, 8, 9, 10, 17, 18, 19, 22, 70, 71, 77, 78, 79, 80, 81, 87, 88, 89, 90, 91, 97, 98, 99, 100, 107, 108, 109, 122, 170, 177, 178, 179, 180, 187, 188, 189, 190, 197, 198, 199, 202, 212, 220, 221, 227, 228, 229, 272, 282, 292, 333, 700, 701, 707, 708, 709, 710, 717, 718, 719, 722, 770, 771, 777, 778, 779, 780, 781, 787, 788, 789, 790, 791, 797, 798, 799, 800, 801, 807, 808, 809, 810, 817, 818, 819, 822, 870, 871, 877, 878, 879, 880, 881, 887, 888, 889, 890, 891, 897, 898, 899, 900, 901, 907, 908, 909, 910, 917, 918, 919, 922, 970, 971, 977, 978, 979, 980, 981, 987, 988, 989, 990, 991, 997, 998, 999, 1000, 1007, 1008, 1009, 1022, 1070, 1077, 1078, 1079, 1080, 1087, 1088, 1089, 1090, 1097, 1098, 1099, 1202, 1220, 1227, 1228, 1229, 1272, 1282, 1292, 1333, 1700, 1707, 1708, 1709, 1722, 1770, 1777, 1778, 1779, 1780, 1787, 1788, 1789, 1790, 1797, 1798, 1799, 1800, 1807, 1808, 1809, 1822, 1870, 1877, 1878, 1879, 1880, 1887, 1888, 1889, 1890, 1897, 1898, 1899, 1900, 1907, 1908, 1909, 1922,

In [3]:
mean_nums_count = []

def check(nums):
    for i, n in enumerate(nums):
        if n == 0:
            continue
        if i != n:
            return False
    return True

def mean_nums(num):
    res = [0] * 10
    while num:
        x = num % 10
        res[x] += 1
        num = num // 10
    return res

# 优化：减少循环范围
for i in range(10000000):
    if i > 1000000:  # 假设超过 1000000 的数字不可能满足条件
        break
    tmp = mean_nums(i)
    if check(tmp):
        mean_nums_count.append(i)

print(mean_nums_count)

[0, 1, 22, 122, 212, 221, 333, 1333, 3133, 3313, 3331, 4444, 14444, 22333, 23233, 23323, 23332, 32233, 32323, 32332, 33223, 33232, 33322, 41444, 44144, 44414, 44441, 55555, 122333, 123233, 123323, 123332, 132233, 132323, 132332, 133223, 133232, 133322, 155555, 212333, 213233, 213323, 213332, 221333, 223133, 223313, 223331, 224444, 231233, 231323, 231332, 232133, 232313, 232331, 233123, 233132, 233213, 233231, 233312, 233321, 242444, 244244, 244424, 244442, 312233, 312323, 312332, 313223, 313232, 313322, 321233, 321323, 321332, 322133, 322313, 322331, 323123, 323132, 323213, 323231, 323312, 323321, 331223, 331232, 331322, 332123, 332132, 332213, 332231, 332312, 332321, 333122, 333212, 333221, 422444, 424244, 424424, 424442, 442244, 442424, 442442, 444224, 444242, 444422, 515555, 551555, 555155, 555515, 555551, 666666]


In [15]:
import time
def custom_divmod(n, D):
    if D == 0:
        raise ZeroDivisionError("division by zero")
    
    # 计算商
    w = n // D
    # 计算余数
    r = n % D
    
    return w, r
# 测试数据
test_cases = [
    (10, 3),
    (-10, 3),
    (10.5, 3),
    (10, 0),  # 测试除以零的情况
]

# 比较功能正确性
print("比较功能正确性：")
for n, D in test_cases:
    try:
        w_builtin, r_builtin = divmod(n, D)
        w_custom, r_custom = custom_divmod(n, D)
        print(f"n={n}, D={D}")
        print(f"内置 divmod: 商={w_builtin}, 余数={r_builtin}")
        print(f"自定义 divmod: 商={w_custom}, 余数={r_custom}")
        print("是否一致:", w_builtin == w_custom and r_builtin == r_custom)
    except ZeroDivisionError as e:
        print(f"n={n}, D={D} - 错误: {e}")
    print()

# 比较性能
print("比较性能：")
n = 10000000
D = 3

# 内置 divmod
start_time = time.time()
for _ in range(n):
    divmod(10, 3)
end_time = time.time()
builtin_time = end_time - start_time
print(f"内置 divmod 执行时间: {builtin_time:.6f} 秒")

# 自定义 divmod
start_time = time.time()
for _ in range(n):
    custom_divmod(10, 3)
end_time = time.time()
custom_time = end_time - start_time
print(f"自定义 divmod 执行时间: {custom_time:.6f} 秒")

# 比较性能差异
print(f"性能差异: {custom_time / builtin_time:.2f} 倍")

比较功能正确性：
n=10, D=3
内置 divmod: 商=3, 余数=1
自定义 divmod: 商=3, 余数=1
是否一致: True

n=-10, D=3
内置 divmod: 商=-4, 余数=2
自定义 divmod: 商=-4, 余数=2
是否一致: True

n=10.5, D=3
内置 divmod: 商=3.0, 余数=1.5
自定义 divmod: 商=3.0, 余数=1.5
是否一致: True

n=10, D=0 - 错误: integer division or modulo by zero

比较性能：
内置 divmod 执行时间: 0.754683 秒
自定义 divmod 执行时间: 1.308909 秒
性能差异: 1.73 倍
